In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：用于批量预测的AutoML视频动作识别模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_video_action_recognition_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_video_action_recognition_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库创建视频动作识别模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)进行批量预测。

### 数据集

本教程使用的数据集是来自[麻省理工学院](http://cbcl.mit.edu/publications/ps/Kuehne_etal_iccv11.pdf)的[人体运动数据集](https://todo)中的高尔夫挥杆识别部分。您在本教程中将使用的数据集版本存储在公共云存储桶中。训练后的模型将预测高尔夫挥杆开始的起始帧。

### 目标

在本教程中，您将从Python脚本中创建一个AutoML视频动作识别模型，然后使用Vertex客户端库进行批量预测。您还可以选择使用`gcloud`命令行工具或在Google Cloud控制台上在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

在使用批量预测和在线预测之间有一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项）进行按需预测，并实时返回结果。

* 批量预测服务：对整个实例集进行排队（批量）预测，并在准备好时将结果存储在Cloud Storage存储桶中。

### 成本

本教程使用谷歌云的计费组件（GCP）：

- Vertex AI
- 云存储

了解[Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)和[云存储定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)，根据您的预期使用情况生成成本估算。

## 安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = '--user'
else:
    USER_FLAG = ''

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始之前

###GPU运行时

*如果可以选择，确保在GPU运行时中运行此笔记本。在Colab中，选择* **Runtime > Change Runtime Type > GPU**

###设置您的谷歌云项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额，可用于支付计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用Vertex APIs和Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk)已经安装在Google Cloud Notebook中。

5. 在下面的单元格中输入您的项目ID。然后运行单元格，确保云SDK在此笔记本中的所有命令中使用正确的项目。 

**注意**：Jupyter将带有`!`前缀的行视为shell命令，并将以`$`前缀的Python变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  #@param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改 `REGION` 变量，该变量用于笔记本其余部分的操作。以下是Vertex支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法在Vertex上使用多区域存储桶进行训练。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持情况，请参阅[Vertex位置文档] (https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = 'us-central1'  #@param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能会使用共享的测试账户或项目。为了避免用户之间在创建的资源上发生名称冲突，您需要为每个实例会话创建一个时间戳，并附加到将在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用谷歌云笔记本**，您的环境已经通过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证以通过oAuth验证您的账户。

**否则**，请按照以下步骤操作：

在云控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐号访问项目**部分，点击角色下拉列表。在过滤框中输入“Vertex”，选择**Vertex管理员**。在过滤框中输入“Storage Object Admin”，选择**Storage Object Admin**。

点击创建。包含您密钥的JSON文件将下载到您的本地环境中。

在下面的单元格中，将您的服务帐号密钥的路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，都需要执行以下步骤。**

本教程旨在使用位于公共云存储桶中的训练数据以及本地云存储桶进行批量预测。您也可以使用存储在本地云存储桶中的自己的训练数据。

在下面设置您的云存储桶的名称。存储桶名称必须在所有谷歌云项目中全局唯一，包括您组织外的那些项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  #@param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时才执行下面的单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库和定义常量

#### 导入 Vertex 客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点API服务的数据集、模型、作业、流水线和终节点服务终点。
- `PARENT`：顶点资源的数据集、模型、作业、流水线和终节点的位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### 自动机器学习常量

设置独特于自动机器学习数据集和训练的常量：

- 数据集模式：告诉`数据集`资源服务数据集的类型是什么。
- 数据标注模式：告诉`数据集`资源服务数据的标注方式。
- 数据集训练模式：告诉`管道`资源服务为哪个任务（例如分类）训练模型。

In [ ]:
# Video Dataset type
DATA_SCHEMA = 'gs://google-cloud-aiplatform/schema/dataset/metadata/video_1.0.0.yaml'
# Video Labeling type
LABEL_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/ioformat/video_action_recognition_io_format_1.0.0.yaml"
# Video Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_video_action_recognition_1.0.0.yaml"

硬件加速器

设置硬件加速器（例如 GPU），如果有的话，用于预测。

将变量 `DEPLOY_GPU/DEPLOY_NGPU` 设置为使用支持 GPU 的容器映像以及分配给虚拟机实例的 GPU 数量。例如，要使用一个 GPU 容器映像，并为每个 VM 分配了 4 个 Nvidia Telsa K80 GPU，您将指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于 GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定 `(None, None)` 使用一个容器映像在 CPU 上运行。

In [ ]:
if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, int(os.getenv("IS_TESTING_DEPOLY_GPU")))
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

Docker容器（Docker）映像

对于AutoML批量预测，由Vertex预测服务预先确定服务二进制文件的容器映像。更具体地，该服务将根据您选择的硬件加速器为模型选择适当的容器。

#### 机器类型

接下来，设置用于预测的机器类型。

- 将变量 `DEPLOY_COMPUTE` 设置为配置用于预测的虚拟机的计算资源。
  - `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 的内存。
     - `n1-highmem`: 每个 vCPU 6.5GB 的内存
     - `n1-highcpu`: 每个 vCPU 0.9 GB 的内存
  - `vCPUs`: 数量为 \[2, 4, 8, 16, 32, 64, 96 \]

*注意：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = 'n1-standard'

VCPU = '4'
DEPLOY_COMPUTE = MACHINE_TYPE + '-' + VCPU
print('Deploy machine type', DEPLOY_COMPUTE)

# 教程

现在你已经准备好开始创建你自己的AutoML视频动作识别模型。

## 设置客户端

Vertex客户端库采用客户端/服务器模型。在您的端上（Python脚本），您将创建一个客户端，向Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流程的不同步骤使用不同的客户端。因此，请提前设置好它们。

- 数据集服务用于管理`数据集`资源。
- 模型服务用于管理`模型`资源。
- 管道服务用于训练。
- 作业服务用于批量预测和自定义训练。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(
        client_options=client_options
    )
    return client


def create_model_client():
    client = aip.ModelServiceClient(
        client_options=client_options
    )
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(
        client_options=client_options
    )
    return client


def create_job_client():
    client = aip.JobServiceClient(
        client_options=client_options
    )
    return client


clients = {}
clients['dataset'] = create_dataset_client()
clients['model'] = create_model_client()
clients['pipeline'] = create_pipeline_client()
clients['job'] = create_job_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户已经准备就绪，训练模型的第一步是创建一个托管的数据集实例，然后将已标记的数据上传到其中。

### 创建`Dataset`资源实例

使用辅助函数`create_dataset`来创建`Dataset`资源的实例。此函数执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个Vertex `Dataset`资源（`aip.Dataset`），具有以下参数：
 - `display_name`：您选择的易读名称。
 - `metadata_schema_uri`：数据集类型的模式。
3. 调用客户端数据集服务方法`create_dataset`，具有以下参数：
 - `parent`：用于指定您的`Database`、`Model`和`Endpoint`资源的Vertex位置根路径。
 - `dataset`：您创建的Vertex数据集对象实例。
4. 该方法返回一个`operation`对象。

`operation`对象是Vertex处理异步调用长时间操作的方式。虽然这一步通常很快，但在项目中首次使用时，由于需进行供应，会有较长的延迟。

您可以使用`operation`对象来获取操作的状态（例如，创建`Dataset`资源）或取消操作，通过调用操作方法：

| 方法         | 描述           |
| ----------- | ----------- |
| result()    | 等待操作完成并以JSON格式返回结果对象。      |
| running()   | 返回操作是否仍在运行的True/False。        |
| done()      | 返回操作是否已完成的True/False。 |
| canceled()  | 返回操作是否已取消的True/False。 |
| cancel()    | 取消操作（可能需要最多30秒）。 |

In [ ]:
TIMEOUT = 90

def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(display_name=name, metadata_schema_uri=schema, labels=labels)

        operation = clients['dataset'].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("golf-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的“Dataset”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split('/')[-1]

print(dataset_id)

### 数据准备

Vertex视频的`Dataset`资源对您的数据有一些要求。

- 视频必须存储在云存储桶中。
- 每个视频文件必须是视频格式（MPG，AVI，...）。
- 在您的云存储桶中必须有一个索引文件，其中包含每个视频的路径和标签。
- 索引文件必须是CSV或JSONL格式。

CSV

对于视频动作识别，CSV索引文件有一些要求：

- 没有表头。
- 第一列是视频的Cloud Storage路径。
- 第二列是要分析的视频片段的开始时间偏移量。
- 第三列是要分析的视频片段的结束时间偏移量。
- 第四列是动作的标签（例如：挥动）。
- 第五列是已识别动作的时间偏移量。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILES = ['gs://automl-video-demo-data/hmdb_golf_swing_train.csv', 'gs://automl-video-demo-data/hmdb_golf_swing_test.csv']

快速查看您的数据

您将使用存储在公共云存储桶中的高尔夫挥杆数据集的一个版本，使用CSV索引文件。

先从快速查看数据开始。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行数据。

In [ ]:
if 'IMPORT_FILES' in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的 Vertex Dataset 资源中。使用这个辅助函数 `import_data` 来导入数据。该函数会执行以下操作：

- 使用 `Dataset` 客户端。
- 调用客户端方法 `import_data`，并提供以下参数：
  - `name`：您对 `Dataset` 资源命名的可读名称（例如，golf）。
  - `import_configs`：导入配置。

- `import_configs`：一个包含字典的 Python 列表，其中包含键/值条目：
  - `gcs_sources`：包含一个或多个索引文件路径的 URI 列表。
  - `import_schema_uri`：标识标签类型的模式。

`import_data()` 方法会返回一个长时间运行的 `operation` 对象。这需要几分钟才能完成。如果您正在参加现场教程，现在是提问或休息的好时机。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{
        'gcs_source': {'uris': gcs_sources},
        'import_schema_uri': schema
    }]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients['dataset'].import_data(name=dataset_id, import_configs=config)
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print("after: running:", operation.running(), "done:", operation.done(), "cancelled:", operation.cancelled())

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, IMPORT_FILES, LABEL_SCHEMA)

## 训练模型

现在使用您的 Vertex `Dataset` 资源来训练一个 AutoML 视频动作识别模型。要训练模型，请执行以下步骤：

1. 为 `Dataset` 资源创建一个 Vertex 训练流水线。
2. 执行流水线以开始训练。

### 创建一个训练管道

您可能会问，我们使用管道做什么？通常在工作（如训练）有多个步骤时，您会使用管道，这些步骤通常是按顺序进行的：执行步骤A，执行步骤B，依此类推。将这些步骤放入管道中，我们可以获得以下好处：

1. 可以重复使用于后续的训练工作。
2. 可以将其封装为容器并作为批处理任务运行。
3. 可以进行分布式处理。
4. 所有步骤都与同一管道工作相关联，以便跟踪进度。

使用这个辅助函数 `create_pipeline`，它接受以下参数：

- `pipeline_name`：管道工作的人类可读名称。
- `model_name`：模型的人类可读名称。
- `dataset`：Vertex 完全限定的数据集标识符。
- `schema`：数据集标签（注释）训练模式。
- `task`：描述训练工作要求的字典。

该辅助函数调用 `Pipeline` 客户端服务的方法 `create_pipeline`，它接受以下参数：

- `parent`：您的 `Dataset`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
- `training_pipeline`：管道训练工作的完整规范。

现在让我们更深入地看一下构建 `training_pipeline` 规范的*最低要求*：

- `display_name`：管道工作的人类可读名称。
- `training_task_definition`：数据集标签（注释）训练模式。
- `training_task_inputs`：描述训练工作要求的字典。
- `model_to_upload`：模型的人类可读名称。
- `input_data_config`：数据集规范。
  - `dataset_id`：仅包含 Vertex 数据集标识符（非完全限定）-- 这是完全限定标识符的最后部分。
  - `fraction_split`：如果指定了，数据集用于训练、测试和验证的百分比。否则，百分比将由 AutoML 自动选择。
  - 注意：对于视频，不支持验证分割-- 只支持训练和测试。

In [ ]:
 def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split('/')[-1]

    input_config = {'dataset_id': dataset_id,
                    'fraction_split': {
                        'training_fraction': 0.8,
                        'test_fraction': 0.2
                    }}

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients['pipeline'].create_training_pipeline(parent=PARENT, training_pipeline=training_pipeline)
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与其他参数不同，需要一个谷歌 protobuf 结构来指定任务，而不是像使用 Python（类似 JSON）字典那样。使用 `json_format.ParseDict` 方法进行转换。

您需要指定的最小字段包括：

- `model_type`：部署模型的类型，例如部署到谷歌云的 CLOUD。

最后，通过调用帮助函数 `create_pipeline` 来创建流水线，该函数将返回一个训练流水线对象的实例。

In [ ]:
PIPE_NAME = "golf_pipe-" + TIMESTAMP
MODEL_NAME = "golf_model-" + TIMESTAMP

task = json_format.ParseDict({'model_type': "CLOUD",
                             }, Value())

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练管道的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split('/')[-1]

print(pipeline_id)

### 获取培训流水线的信息

现在只需获取此培训流水线实例的流水线信息。通过调用作业客户服务的`get_training_pipeline` 方法来获取此作业的作业信息，使用以下参数：

- `name`：Vertex 完全合格的流水线标识符。

当模型训练完成时，流水线状态将为`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients['pipeline'].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

# 部署

训练以上模型可能需要高达240分钟的时间。

一旦您的模型训练完成，您可以通过将 `end_time` 减去 `start_time` 来计算训练模型所花费的实际时间。对于您的模型，您需要知道完全限定的 Vertex 模型资源标识符，这是管道服务分配给它的。您可以从返回的管道实例中获取此信息，作为字段 `model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

模型信息

现在你的模型已经训练好了，你可以获取一些关于你的模型的信息。

## 评估模型资源

现在找出模型服务认为你的模型有多好。作为训练的一部分，数据集的一部分被保留为测试（留存）数据，管道服务使用该数据来评估模型。

### 列出所有切片的评估

使用此辅助函数`list_model_evaluations`，它接受以下参数：

- `name`：`Model` 资源的 Vertex 完全限定模型标识符。

此辅助函数使用模型客户端服务的`list_model_evaluations`方法，该方法接受相同的参数。调用的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估 -- 您可能只有一个，然后打印出评估中每个指标的所有键名称，对于一小组（`videoActionMetrics`），您将打印结果。

In [ ]:
def list_model_evaluations(name):
    response = clients['model'].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print('videoActionMetrics', metrics['videoActionMetrics'])


    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

## 批量预测模型部署

现在部署你为批量预测创建的训练过的 Vertex `Model` 资源。这与为按需预测部署 `Model` 资源不同。

对于在线预测，你需要：

1. 创建一个用于部署 `Model` 资源的 `Endpoint` 资源。

2. 将 `Model` 资源部署到 `Endpoint` 资源上。

3. 向 `Endpoint` 资源发起在线预测请求。

对于批量预测，你需要：

1. 创建一个批量预测作业。

2. 作业服务将为批量预测请求分配资源。

3. 返回批量预测请求的结果给调用方。

4. 作业服务将取消为批量预测请求分配的资源。

发起一个批量预测请求

现在对您部署的模型进行批量预测。

获取测试项目

现在对您的Vertex模型进行批量预测。您将使用数据集中的任意示例作为测试项目。不要担心这些示例可能已经在训练模型中使用过--我们只是想演示如何进行预测。

In [ ]:
import json

import_file = IMPORT_FILES[0]
test_items = ! gsutil cat $import_file | head -n2

cols = str(test_items[0]).split(',')
test_item_1 = str(cols[0])
test_label_1 = str(cols[-1])

cols = str(test_items[1]).split(',')
test_item_2 = str(cols[0])
test_label_2 = str(cols[-1])

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

制作批量输入文件

现在制作一个批量输入文件，将其存储在您的本地云存储桶中。批量输入文件可以是CSV或JSONL格式。在本教程中，您将使用JSONL格式。对于JSONL文件，您每行为每个视频创建一个字典条目。字典包含键/值对：

- `content`：视频的云存储路径。
- `mimeType`：内容类型。在我们的示例中，是一个`avi`文件。
- `timeSegmentStart`：要在视频上进行预测的开始时间戳。*注意*，时间戳必须指定为字符串，后跟s（秒）、m（分钟）或h（小时）。
- `timeSegmentEnd`：要在视频上进行预测的结束时间戳。

In [ ]:
import json

import tensorflow as tf

gcs_input_uri = BUCKET_NAME + '/test.jsonl'
with tf.io.gfile.GFile(gcs_input_uri, 'w') as f:
    data = { "content": test_item_1, "mimeType": "video/avi", "timeSegmentStart": "0.0s", 'timeSegmentEnd': '5.0s' }
    f.write(json.dumps(data) + '\n')
    data = { "content": test_item_2, "mimeType": "video/avi", "timeSegmentStart": "0.0s", 'timeSegmentEnd': '5.0s' }
    f.write(json.dumps(data) + '\n')

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 计算实例扩展

处理批量预测请求时，您有几种计算实例扩展的选择：

- 单个实例：批量预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：批量预测请求在您手动指定的固定数量的计算实例上分配。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同数量的节点。当模型首次部署到实例时，会提供固定数量的计算实例，并且批量预测请求会均匀分布在这些实例上。

- 自动扩展：批量预测请求在可扩展数量的计算实例上分配。
  - 将计算实例的最小（`MIN_NODES`）数量设置为在模型首次部署时提供的实例数量，并且在负载情况下取消提供，将最大（`MAX_NODES`）计算实例数量设置为根据负载条件提供。

计算实例的最小数量对应于字段`min_replica_count`，最大数量对应于字段`max_replica_count`，在您的后续部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 发出批量预测请求

现在您的两个测试项的批量已经准备好了，让我们进行批量请求。使用这个辅助函数 `create_batch_prediction_job`，并提供以下参数：

- `display_name`: 预测任务的人类可读名称。
- `model_name`: 指向 `Model` 资源的 Vertex 完全限定标识符。
- `gcs_source_uri`: 输入文件在 Cloud Storage 中的路径，这是您之前创建的路径。
- `gcs_destination_output_uri_prefix`: 服务将把预测结果写入的 Cloud Storage 路径。
- `parameters`: 用于过滤服务端预测结果的额外参数。

这个辅助函数调用作业客户端服务的 `create_batch_prediction_job` 方法，并提供以下参数：

- `parent`: Dataset、Model 和 Pipeline 资源的 Vertex 位置根路径。
- `batch_prediction_job`: 批量预测任务的规范。

现在让我们深入了解 `batch_prediction_job` 的规范：

- `display_name`: 预测批处理任务的人类可读名称。
- `model`: 指向 `Model` 资源的 Vertex 完全限定标识符。
- `dedicated_resources`: 用于为批量预测任务分配的计算资源。
  - `machine_spec`: 要分配的计算实例。根据之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只会分配 CPU。
  - `starting_replica_count`: 最初要分配的计算实例数量，您之前设置为变量 `MIN_NODES`。
  - `max_replica_count`: 可扩展到的最大计算实例数量，您之前设置为变量 `MAX_NODES`。
- `model_parameters`: 用于服务端预测结果的额外过滤参数。
 - `confidenceThreshold`: 进行预测的最小置信度阈值。
 - `maxPredictions`: 每个操作返回的最大预测数量，按置信度排序。
- `input_config`: 用于进行预测的实例的输入源和格式类型。
 - `instances_format`: 批量预测请求文件的格式：`csv` 或 `jsonl`。
 - `gcs_source`: 包含一个或多个 Cloud Storage 路径来访问批量预测请求。
- `output_config`: 预测结果的输出目的地和格式。
 - `prediction_format`: 批量预测响应文件的格式：仅为 `jsonl`。
 - `gcs_destination`: 预测结果的输出目的地。

您可能会问，置信度阈值如何影响模型准确性？这个阈值不会改变准确性，它会改变召回率和精确率。

    - 精确率：精确率越高，所预测的结果越有可能是正确的，但会返回较少的预测。增加置信度阈值会提高精确率。
    - 召回率：召回率越高，结果中返回正确预测的可能性越大，但会返回更多的带有不正确预测的结果。减少置信度阈值会提高召回率。

在这个示例中，您将预测精确度。您将把置信度阈值设置为 0.5，每次操作的最大预测数量为两个。由于所有类别的置信度值必须相加为一，只有两种可能的结果：

    1. 存在 tie，均为 0.5，返回两个预测。
    2. 一个值高于 0.5，其余值低于 0.5，返回一个预测。

此调用是一个异步操作。您将从响应对象中打印几个选择的字段，包括：

- `name`: 分配给批量预测任务的 Vertex 完全限定标识符。
- `display_name`: 预测批处理任务的人类可读名称。
- `model`: `Model` 资源的 Vertex 完全限定标识符。
- `generate_explanations`: 预测结果是否提供了 True/False 的解释（解释性）。
- `state`: 预测任务的状态（等待中、运行中等）。

由于这个调用需要一些时间来执行，您可能会在 `state` 中得到 `JobState.JOB_STATE_PENDING`。

In [ ]:
BATCH_MODEL = "golf_batch-" + TIMESTAMP


def create_batch_prediction_job(display_name, model_name, gcs_source_uri, gcs_destination_output_uri_prefix, parameters=None):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    batch_prediction_job = {
        "display_name": display_name,
        # Format: 'projects/{project}/locations/{location}/models/{model_id}'
        "model": model_name,
        "model_parameters": json_format.ParseDict(parameters, Value()),
        "input_config": {
            "instances_format": IN_FORMAT,
            "gcs_source": {"uris": [gcs_source_uri]},
        },
        "output_config": {
            "predictions_format": OUT_FORMAT,
            "gcs_destination": {"output_uri_prefix": gcs_destination_output_uri_prefix},
        },
        "dedicated_resources": {
            "machine_spec": machine_spec,
            "starting_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES
        }

    }
    response = clients['job'].create_batch_prediction_job(
        parent=PARENT, batch_prediction_job=batch_prediction_job
    )
    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", response.labels)
    return response


IN_FORMAT = 'jsonl'
OUT_FORMAT = 'jsonl'  # [jsonl]

response = create_batch_prediction_job(BATCH_MODEL, model_to_deploy_id, gcs_input_uri, BUCKET_NAME,
                                       {'confidenceThreshold': 0.5, 'maxPredictions': 2})

现在获得您创建的批量预测作业的唯一标识符。

In [ ]:
# The full unique ID for the batch job
batch_job_id = response.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split('/')[-1]

print(batch_job_id)

### 获取批量预测作业的信息

使用这个辅助函数`get_batch_prediction_job`，并传入以下参数:

- `job_name`：批量预测作业的Vertex完全限定标识符。

辅助函数会调用作业客户端服务的`get_batch_prediction_job`方法，并传入以下参数:

- `name`：批量预测作业的Vertex完全限定标识符。在本教程中，您将传递给它您的批量预测作业的Vertex完全限定标识符 -- `batch_job_id`

辅助函数将返回存储预测结果的Cloud Storage路径 -- `gcs_destination`。

In [ ]:
def get_batch_prediction_job(job_name, silent=False):
    response = clients['job'].get_batch_prediction_job(name=job_name)
    if silent:
        return response.output_config.gcs_destination.output_uri_prefix, response.state

    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:   # not all data types support explanations
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" error:", response.error)
    gcs_destination = response.output_config.gcs_destination
    print(" gcs_destination")
    print("  output_uri_prefix:", gcs_destination.output_uri_prefix)
    return gcs_destination.output_uri_prefix, response.state


predictions, state = get_batch_prediction_job(batch_job_id)

### 获取预测结果

当批量预测处理完成时，作业状态将会是 `JOB_STATE_SUCCEEDED`。

最终您可以查看存储在您设置为输出的 Cloud Storage 路径中的预测结果。预测结果将以 JSONL 格式存储，在您创建批量预测作业时指定的路径下，以以`prediction`开头的子文件夹，该文件夹下将会有一个名为 `predictions*.jsonl` 的文件。

现在显示（cat）其中的内容。您将看到多个 JSON 对象，每个对象对应一个预测结果。

对于每个预测结果：

- `content`：用于预测请求的视频。
- `displayName`：预测动作。
- `confidence`：预测的置信度，介于 0 和 1 之间。
- `timeSegmentStart/timeSegmentEnd`：预测动作的开始和结束的时间偏移。

In [ ]:
def get_latest_predictions(gcs_out_dir):
    ''' Get the latest prediction subfolder using the timestamp in the subfolder name'''
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split('/')[-2]
        if subfolder.startswith('prediction-'):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    predictions, state = get_batch_prediction_job(batch_job_id, True)
    if state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", state)
        if state == aip.JobState.JOB_STATE_FAILED:
            raise Exception("Batch Job Failed")
    else:
        folder = get_latest_predictions(predictions)
        ! gsutil ls $folder/prediction*.jsonl

        ! gsutil cat $folder/prediction*.jsonl
        break
    time.sleep(60)

清理工作

要清理本项目中使用的所有 GCP 资源，您可以[删除用于本教程的 GCP 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and 'dataset_id' in globals():
        clients['dataset'].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and 'pipeline_id' in globals():
        clients['pipeline'].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and 'model_to_deploy_id' in globals():
        clients['model'].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and 'endpoint_id' in globals():
        clients['endpoint'].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and 'batch_job_id' in globals():
        clients['job'].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and 'job_id' in globals():
        clients['job'].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and 'hpt_job_id' in globals():
        clients['job'].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil rm -r $BUCKET_NAME